In [1]:
class Meta_Search_Optimizer:
    
    '''Meta Search Optimizer
       
       @init:
       max_steps = max number of iterations
       verbose = True shows progress bar
       
       @optimize:
           *input:
               f = function to be optimized
               warning: 
                   - f.opt = optimal value
                   - f.domain = interval to search, e.g. [-5, 5]
                   - f.dimension = dimension of the function
           *output:
               (best_x, number of steps) if optimal point was found else (None, -1)
    '''
    
    def __init__(self, meta_model, init_hf_strategy="zero", best_meta_point_strategy="lowest", 
                 hf_strategy="random", nb_samples=10, size_hf=5, max_steps=100000, verbose=False):
        self.meta_model = meta_model
        self.init_hf_strategy = init_hf_strategy
        self.best_meta_point_strategy = best_meta_point_strategy
        self.hf_strategy = hf_strategy
        self.nb_samples = nb_samples
        self.size_hf = size_hf
        self.max_steps = max_steps
        self.verbose = verbose
        
    def init_hf(self):
        if self.init_hf_strategy == "zero":
            return np.array([(0,0,0)]*self.size_hf)
        
    def best_meta_point(self, meta_points, meta_evaluations, hf):
        if self.best_meta_point_strategy == "lowest":
            meta_evaluations = meta_evaluations.reshape(self.nb_samples)
            indexes = np.argsort(meta_evaluations)
            hf = np.array(hf)
            if hf.size == 0:
                return meta_points[indexes[0]]
            for idx in indexes:
                if meta_points[idx] not in hf[:, 0:2]:
                    return meta_points[idx]
            return meta_points[indexes[0]]
        if self.best_meta_point_strategy == "random":
            index = np.random.randint(0, len(meta_points))
            return meta_points[index]
        
    def optimize(self, f):
        
        #Init best values:
        best_x = [None] * f.dimension 
        best_y = 99999.0
        
        #Init history
        hf = []
        curr_hf = self.init_hf()
        
        #Define Iterator:
        if self.verbose:
            iterator = tqdm(range(self.max_steps))
        else:
            iterator = range(self.max_steps)
        
        #Iteration:
        for step in iterator:
            
            #1) Select n random points to be evaluated by meta-model
            meta_points = (f.domain[1] - f.domain[0]) * np.random.random_sample((self.nb_samples, f.dimension)) + f.domain[0]
            
            #2) Meta-evaluate points in meta-model
            meta_evaluations = self.meta_model.predict(meta_points, curr_hf)
            
            #3) Get the next point (x_t+1)
            new_x = self.best_meta_point(meta_points, meta_evaluations, hf)

            #4) Evaluate this point in function
            new_y = f(new_x)
            
            if step % 1000 == 0:
                
                print(meta_points)
                print(meta_evaluations)
                print(new_x)
                print(new_y)
                print()
            
            #Check if it is an improvement:
            if new_y < best_y: 
                best_y = new_y
                best_x = new_x
                
                #Check if it is a solution:
                if round(best_y, 2) == f.fopt:
                    return (list(best_x), step)

            #Save train data:
            #dataset.append(np.array([np.array(best_point), np.array(curr_hist), np.array(f_eval)]))

            #5) Add this info in your history
            info = (new_x[0], new_x[1], new_y)  #TODO: here it's only 2D problem!
            hf.append(info)

            #6) Select next current history
            curr_hf = self.select_hf(hf)
            
        return (None, -1)
            
    def select_hf(self, hf):

        if self.hf_strategy == "gagne":
            
            if len(hf) < self.size_hf:
                choices = np.random.choice(len(hf), self.size_hf)
                curr_pop = np.array([hf[i] for i in choices])
                return curr_pop
            else:
                #50% top:
                top50 = sorted(range(len(hf)), key=lambda i: hf[i], reverse=True)[-self.size_hf//2:]

                #50% random:
                random50 = [random.randint(0, len(hf)-1) for i in range(self.size_hf//2)]

                all_indexes = sorted(top50 + random50)
                curr_pop = [(hf[ind][0], hf[ind][1], hf[ind][2]) for ind in all_indexes]

                return np.array(curr_pop)

        elif self.hf_strategy == "random":
            hf = np.array(hf)
            idx = np.random.choice(len(hf), self.size_hf)
            return hf[idx]

In [2]:
class Random_Search_Optimizer:
    
    '''Random Search Optimizer
       
       @init:
       max_steps = max number of iterations
       verbose = True shows progress bar
       
       @optimize:
           *input:
               f = function to be optimized
               warning: 
                   - f.opt = optimal value
                   - f.domain = interval to search, e.g. [-5, 5]
                   - f.dimension = dimension of the function
           *output:
               (best_x, number of steps) if optimal point was found else (None, -1)
    '''
    
    def __init__(self, max_steps=100000, verbose=False):
        self.max_steps = max_steps
        self.verbose = verbose
    
    def optimize(self, f):
        
        #Init best values:
        best_x = [None] * f.dimension 
        best_y = 99999.0

        #Define Iterator:
        if self.verbose:
            iterator = tqdm(range(self.max_steps))
        else:
            iterator = range(self.max_steps)
        
        #Iteration:
        for step in iterator:

            #Choose random x and evaluate it:
            new_x = (f.domain[1] - f.domain[0]) * np.random.random_sample(f.dimension) + f.domain[0]
            new_y = f(new_x)

            #Check if it is an improvement:
            if new_y < best_y: 
                best_y = new_y
                best_x = new_x

                #Check if it is a solution:
                if round(best_y, 2) == f.fopt:
                    return (list(best_x), step)

        return (None, -1)

In [3]:
class Evaluator:
    
    '''Evaluator
       
       @init:
       nb_trials = max number of iterations
       verbose = True shows progress bar
       
       @optimize:
           *input:
               f = function to be optimized
               warning: 
                   - f.opt = optimal value
                   - f.domain = interval to search, e.g. [-5, 5]
                   - f.dimension = dimension of the function
                optimizer = optimize that will be used to optimize the function
           *output:
               if optimal point was found in all iterations: 
                   {"mean": mean, "stdev": stdev}
               else: 
                   {"mean": None, "stdev": None}
    '''
    
    def __init__(self, nb_trials=100, verbose=1):
        self.nb_trials = nb_trials
        self.verbose = verbose

    def evaluate(self, f, optimizer):
        
        sum_x = 0
        sum_x2 = 0
        
        #Define Iterator:
        if self.verbose:
            iterator = tqdm(range(self.nb_trials))
        else:
            iterator = range(self.nb_trials)
        
        #Iteration:
        for trial in iterator:
            bestx, step = optimizer.optimize(f)
            try:
                bestx, step = optimizer.optimize(f)
            except:
                return {"mean": None, "stdev": None}
            
            sum_x += step
            sum_x2 += step*step

        mean = sum_x / self.nb_trials
        stdev = np.sqrt((sum_x2 / self.nb_trials) - (mean * mean)) 
        
        return {"mean": mean, "stdev": stdev}

In [4]:
import numpy as np
from tqdm import tqdm

import bbobbenchmarks as bn

import nbimporter
from metamodel import *

#Define problem
f = bn.F1(1)
f.domain = [-5, 5]
f.dimension = 2

#Define evaluator
evaluator = Evaluator(nb_trials=10)

Importing Jupyter notebook from metamodel.ipynb


Using TensorFlow backend.


In [ ]:
random_search_opt = Random_Search_Optimizer()

#random_search_opt.optimize(f)
#evaluator.evaluate(f, random_search_opt)

In [ ]:
model = Model()
meta_search_opt_random = Meta_Search_Optimizer(model,best_meta_point_strategy="random")

#meta_search_opt_random.optimize(f)
#evaluator.evaluate(f, meta_search_opt_random)

In [5]:
#from keras.models import load_model
#model = load_model('metamodel.h5')
#model = Model(model)

import random
model = Model()


print(model.model.layers[2].input_shape)
meta_search_opt_highest = Meta_Search_Optimizer(model,best_meta_point_strategy="lowest",hf_strategy="gagne",verbose=True)

meta_search_opt_highest.optimize(f)
#evaluator.evaluate(f, meta_search_opt_highest)

  0%|          | 304/100000 [00:00<01:05, 1517.08it/s]

(None, 128)
[[ 0.55200814 -0.3160299 ]
 [ 2.22402152 -3.21376662]
 [ 4.14901092  0.764746  ]
 [-0.40226887 -1.62810665]
 [-3.69141971 -4.96507593]
 [-1.136193   -0.26590595]
 [-2.66690025  4.00225476]
 [ 1.74956985  4.93922901]
 [-0.01593717 -0.89854811]
 [ 1.00407474 -0.7128927 ]]
[[ 0.00137018]
 [ 0.02272679]
 [-0.04484456]
 [-0.00366687]
 [-0.00108001]
 [ 0.00657178]
 [ 0.01377867]
 [-0.10054215]
 [ 0.00023585]
 [ 0.00454964]]
[1.74956985 4.93922901]
118.88188962936746



  1%|          | 1137/100000 [00:01<01:28, 1114.75it/s]

[[-1.31127707 -0.99609205]
 [ 4.91824832  2.94322377]
 [ 1.53164325  4.35836917]
 [ 0.44063632 -3.02953753]
 [ 3.58520108  1.2064341 ]
 [-1.77779662  2.66850605]
 [-0.47035362 -1.76062128]
 [ 1.86012079  4.12057331]
 [ 1.09296291 -1.31791709]
 [-2.75143995  0.4881348 ]]
[[2.6875715]
 [2.632287 ]
 [2.5889611]
 [2.718871 ]
 [2.635746 ]
 [2.6240635]
 [2.6988587]
 [2.5938773]
 [2.6869156]
 [2.6651506]]
[1.53164325 4.35836917]
111.53253098612392



  2%|▏         | 2062/100000 [00:02<02:06, 776.19it/s] 

...

 11%|█         | 11014/100000 [01:03<08:30, 174.25it/s]

[[ 3.15291082 -0.30633336]
 [-1.25142101 -4.99005238]
 [ 1.50225009 -3.14367997]
 [-1.02180565 -2.46263784]
 [ 0.75807252 -3.71360557]
 [-0.82958358 -3.54967909]
 [ 2.02899824  3.87648714]
 [-2.60406282 -3.56768352]
 [-1.52933465  0.58915512]
 [ 0.53981045  1.98454959]]
[[2.5151997]
 [2.5690753]
 [2.5556383]
 [2.5318053]
 [2.5603514]
 [2.5493865]
 [2.4394126]
 [2.5401995]
 [2.482655 ]
 [2.466352 ]]
[2.02899824 3.87648714]
107.96885962416408



 11%|█▏        | 11314/100000 [01:06<08:42, 169.62it/s]

KeyboardInterrupt: 

 11%|█▏        | 11314/100000 [01:20<10:27, 141.39it/s]